In [1]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

#import eli5
#from eli5.sklearn import PermutationImportance

In [2]:
# 필요한 데이터를 load 하겠습니다.
df_path = 'df_cleaned.csv'
df = pd.read_csv(df_path)

In [3]:
# 이제 다시 train과 test dataset을 분할해줍니다. 위에서 제작해 놓았던 is_test 칼럼을 이용합니다.
dt_train = df.query('is_test==0')
dt_test = df.query('is_test==1')

# 이제 is_test 칼럼은 drop해줍니다.
dt_train.drop(['is_test'], axis = 1, inplace=True)
dt_test.drop(['is_test'], axis = 1, inplace=True)
print(dt_train.shape, dt_test.shape)

(1027436, 28) (9260, 28)


In [4]:
dt_test.head(1)

,번지,본번,부번,아파트명,전용면적,계약일,층,건축년도,도로명,k-전체동수,...,주차대수,좌표X,좌표Y,target,구,동,계약년,계약월,강남여부,신축여부
1027436,658-1,658.0,1.0,개포6차우성,79.97,26,5,1987,언주로 3,8.0,...,262.0,127.05721,37.476763,13250.0,강남구,개포동,2023,7,1,0


In [5]:
# dt_test의 target은 일단 0으로 임의로 채워주도록 하겠습니다.
dt_test['target'] = 0

In [6]:
# 파생변수 제작으로 추가된 변수들이 존재하기에, 다시한번 연속형과 범주형 칼럼을 분리해주겠습니다.
continuous_columns_v2 = []
categorical_columns_v2 = []

for column in dt_train.columns:
    if pd.api.types.is_numeric_dtype(dt_train[column]):
        continuous_columns_v2.append(column)
    else:
        categorical_columns_v2.append(column)

print("연속형 변수:", continuous_columns_v2)
print("범주형 변수:", categorical_columns_v2)

연속형 변수: ['본번', '부번', '전용면적', '계약일', '층', '건축년도', 'k-전체동수', 'k-전체세대수', 'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)', 'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', '건축면적', '주차대수', '좌표X', '좌표Y', 'target', '계약년', '계약월', '강남여부', '신축여부']
범주형 변수: ['번지', '아파트명', '도로명', '구', '동']


In [7]:
# Counting the unique values in each categorical column
unique_counts = {column: dt_train[column].nunique() for column in categorical_columns_v2}

unique_counts


{'번지': 5663, '아파트명': 5780, '도로명': 8023, '구': 25, '동': 328}

In [8]:
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import numpy as np

# Updating categorical_columns_v2 to include the columns to be label encoded
label_encode_cols = [ '아파트명', '도로명', '번지', '동']

# 각 변수에 대한 LabelEncoder를 저장할 딕셔너리
label_encoders = {}

# Update the Label Encoding process to handle new values in dt_test
for col in tqdm(label_encode_cols):
    lbl = LabelEncoder()

    # Combining unique values from both train and test sets and adding an 'Other' category
    all_values = np.union1d(dt_train[col].astype(str), dt_test[col].astype(str))
    all_values = np.append(all_values, 'Other')

    # Fit the label encoder with the extended list of values
    lbl.fit(all_values)

    # Transform train and test data
    # New values in test set are replaced with 'Other'
    dt_train[col] = lbl.transform(dt_train[col].astype(str))
    dt_test[col] = dt_test[col].astype(str).apply(lambda x: x if x in lbl.classes_ else 'Other')
    dt_test[col] = lbl.transform(dt_test[col])

    # Saving the label encoder
    label_encoders[col] = lbl

# 원-핫 인코딩을 위한 칼럼 리스트 업데이트
one_hot_encode_cols = ['구', '강남여부', '신축여부']

# 원-핫 인코딩 수행
dt_train = pd.get_dummies(dt_train, columns=one_hot_encode_cols)
dt_test = pd.get_dummies(dt_test, columns=one_hot_encode_cols)

# 결과 확인
dt_train.head(), dt_test.head()


100%|██████████| 4/4 [00:04<00:00,  1.22s/it]


(     번지     본번   부번  아파트명   전용면적  계약일  층  건축년도   도로명  k-전체동수  ...  구_영등포구  \
 0  4342  658.0  1.0   303  79.97    8  3  1987  5365     8.0  ...   False   
 1  4342  658.0  1.0   303  79.97   22  4  1987  5365     8.0  ...   False   
 2  4342  658.0  1.0   303  54.98   28  5  1987  5365     8.0  ...   False   
 3  4342  658.0  1.0   303  79.97    3  4  1987  5365     8.0  ...   False   
 4  4342  658.0  1.0   303  79.97    8  2  1987  5365     8.0  ...   False   
 
    구_용산구  구_은평구  구_종로구   구_중구  구_중랑구  강남여부_0  강남여부_1  신축여부_0  신축여부_1  
 0  False  False  False  False  False   False    True    True   False  
 1  False  False  False  False  False   False    True    True   False  
 2  False  False  False  False  False   False    True    True   False  
 3  False  False  False  False  False   False    True    True   False  
 4  False  False  False  False  False   False    True    True   False  
 
 [5 rows x 54 columns],
            번지     본번   부번  아파트명      전용면적  계약일   층  건축년도   도로명  k-전체동수 

In [9]:
dt_train.head(1)        # 레이블인코딩이 된 모습입니다.

,번지,본번,부번,아파트명,전용면적,계약일,층,건축년도,도로명,k-전체동수,...,구_영등포구,구_용산구,구_은평구,구_종로구,구_중구,구_중랑구,강남여부_0,강남여부_1,신축여부_0,신축여부_1
0,4342,658.0,1.0,303,79.97,8,3,1987,5365,8.0,...,False,False,False,False,False,False,False,True,True,False


In [10]:
assert dt_train.shape[1] == dt_test.shape[1]          # train/test dataset의 shape이 같은지 확인해주겠습니다.

In [11]:
# Target과 독립변수들을 분리해줍니다.
y_train = dt_train['target']
X_train = dt_train.drop(['target'], axis=1)

# Hold out split을 사용해 학습 데이터와 검증 데이터를 8:2 비율로 나누겠습니다.
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=2023)

In [12]:
# # RandomForestRegressor를 이용해 회귀 모델을 적합시키겠습니다.
model = RandomForestRegressor(n_estimators=5, criterion='squared_error', random_state=1, n_jobs=-1)
model.fit(X_train, y_train)
pred = model.predict(X_val)

In [13]:
# 회귀 관련 metric을 통해 train/valid의 모델 적합 결과를 관찰합니다.
print(f'RMSE test: {np.sqrt(metrics.mean_squared_error(y_val, pred))}')

RMSE test: 5615.166756816414


In [14]:
%%time
X_test = dt_test.drop(['target'], axis=1)

# Test dataset에 대한 inference를 진행합니다.
real_test_pred = model.predict(X_test)

CPU times: total: 15.6 ms
Wall time: 50.5 ms


In [15]:
real_test_pred          # 예측값들이 출력됨을 확인할 수 있습니다.

array([201200., 340000., 271100., ...,  84200.,  83240.,  87800.])

In [16]:
# 앞서 예측한 예측값들을 저장합니다.
preds_df = pd.DataFrame(real_test_pred.astype(int), columns=["target"])
preds_df.to_csv('output.csv', index=False)